In [7]:
!pip install altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import altair as alt
import pandas as pd


In [13]:
alt.data_transformers.enable('default')

airportsData = "airports.csv"

airports = pd.read_csv(airportsData, header = 0)

In [14]:
airportsPerCountry = airports["country"].value_counts().rename_axis('countries').reset_index(name='count')
airportsPerCountry['countries'] = airportsPerCountry['countries'].replace('United States', 'United States of America')

airportsPerCountry = airportsPerCountry.iloc[:100]

In [15]:
box = pd.DataFrame({'x1': [0], 'x2': [1600], 'y1': [0], 'y2': [900]})

bg = alt.Chart(box).mark_rect(fill='#aadaff', stroke='#aadaff').encode(
    alt.X('x1', scale=alt.Scale(domain=(0, 1600)), axis = None),
    alt.Y('y1', scale=alt.Scale(domain=(0, 900)), axis = None),
    x2='x2',
    y2='y2'
)

url = "https://raw.githubusercontent.com/deldersveld/topojson/master/world-countries-sans-antarctica.json"

data_map = alt.topo_feature(url, 'countries1')

baseMap = alt.Chart(data_map).mark_geoshape(stroke='white',fill='grey').encode(alt.Tooltip('properties.name:N', title='Country')).properties(
    width=1500,
    height=900,
    title='Top 100 Countries Based on Number of Airports'
)

topMap = alt.Chart(data_map).mark_geoshape(stroke='white').encode(
    tooltip=[alt.Tooltip('properties.name:N', title='Country'), 
             alt.Tooltip('count:Q', title='Number of Airports')],
    color=alt.Color('count:Q', scale=alt.Scale(scheme='greens'),  title = ["Number of Airports"]),
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(airportsPerCountry, 'countries', ['count'])
).properties(
    width=1000,
    height=600,
)

whole = bg+baseMap+topMap

whole.project('equirectangular').configure_title(fontSize = 25).configure_legend(titleFontSize = 18, gradientThickness = 30, gradientLength = 400, labelFontSize = 16, orient = 'bottom-left', titlePadding = 10)


alt.LayerChart(...)